## Face & Eye Detection using HAAR Cascade Classifiers

In [4]:
import numpy as np
import cv2

# We point OpenCV's CascadeClassifier function to where our 
# classifier (XML file format) is stored
face_classifier = cv2.CascadeClassifier(r'D:\nit_prac\Haarcascades\haarcascade_frontalface_default.xml')

# Load our image then convert it to grayscale
image = cv2.imread(r'D:\nit_prac\Haarcascades\krishna.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Our classifier returns the ROI of the detected face as a tuple
# It stores the top left coordinate and the bottom right coordiantes
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No faces found")

# We iterate through our faces array and draw a rectangle
# over each face in faces
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (127,0,255), 2)
    scale_percent = 50  # reduce size to 50%
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    resized = cv2.resize(image, dim)

    cv2.imshow('Face Detection', resized)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

<>:17: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\shreya\AppData\Local\Temp\ipykernel_30596\895955552.py:17: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


### Let's combine face and eye detection

In [5]:
import cv2

# Load cascades
face_classifier = cv2.CascadeClassifier(r'D:\nit_prac\Haarcascades\haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier(r'D:\nit_prac\Haarcascades\haarcascade_eye.xml')

if face_classifier.empty() or eye_classifier.empty():
    print("Error: Could not load Haar cascade XML file(s)!")
    exit()

# Load image
img = cv2.imread(r'D:\nit_prac\Haarcascades\krishna.jpg')
if img is None:
    print("Error: Could not read the image!")
    exit()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

if len(faces) == 0:
    print("No Face Found")

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (127, 0, 255), 2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    # Detect eyes inside face ROI
    eyes = eye_classifier.detectMultiScale(roi_gray, 1.1, 4, minSize=(30,30))
    for (ex, ey, ew, eh) in eyes:
        # --- Filter unwanted detections ---
        if ey + eh/2 > h/2:   # ignore if eye is below face center
            continue
        if eh > h/2:          # ignore very large detections
            continue

        cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 255, 0), 2)

# 🔽 Resize before showing
scale_percent = 50
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(img, dim)

cv2.imshow('Face & Eye Detection', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Let's make a live face & eye detection, keeping the face inview at all times

In [3]:
import cv2

face_classifier = cv2.CascadeClassifier(r'D:\nit_prac\Haarcascades\haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (127, 0, 255), 2)

    cv2.imshow("Live Face Detection", frame)

    # Close if 'q' is pressed OR window is closed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if cv2.getWindowProperty("Live Face Detection", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()


### Tuning Cascade Classifiers

*ourClassifier*.**detectMultiScale**(input image, **Scale Factor** , **Min Neighbors**)

- **Scale Factor**
Specifies how much we reduce the image size each time we scale. E.g. in face detection we typically use 1.3. This means we reduce the image by 30% each time it’s scaled. Smaller values, like 1.05 will take longer to compute, but will increase the rate of detection.



- **Min Neighbors**
Specifies the number of neighbors each potential window should have in order to consider it a positive detection. Typically set between 3-6. 
It acts as sensitivity setting, low values will sometimes detect multiples faces over a single face. High values will ensure less false positives, but you may miss some faces.  
